<a href="https://colab.research.google.com/github/Maria-Rumki/NLP-Assignments/blob/main/Lab_firstQuiz_Mst_Maria_Rahaman_Rumki_st123835.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import torch, torchtext
from torch import nn
import time
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#make our work comparable if restarted the kernel
SEED = 2422
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

cpu


In [2]:
torch.__version__

'1.13.1+cu116'

In [3]:
torchtext.__version__

'0.14.1'

## Load the given dataset

1. Create a variable to your dataset PATH *example:  ./data/*
2. Load the csv files using pandas 



In [4]:
#DATA_PATH = ""
from google.colab import files
train_data_raw = files.upload()
#test_data_raw = 


Saving train.csv to train.csv


In [5]:
test_data_raw = files.upload()

Saving test.csv to test.csv


In [10]:
train_data_raw = pd.read_csv('test.csv')
train_data_raw

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [11]:
test_data_raw = pd.read_csv('train.csv')
test_data_raw

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [13]:
## Lets analyze the data a little

#print and show how many unique classes are in the target

classes = test_data_raw["target"].unique()
num_classes = len(classes)
classes


array([1, 0])

In [14]:
num_classes

2

In [15]:
assert num_classes > 1

In [17]:
##lets see how many columns are there
#print the columns of the train_data_raw

print(train_data_raw) #write your code here

         id keyword location  \
0         0     NaN      NaN   
1         2     NaN      NaN   
2         3     NaN      NaN   
3         9     NaN      NaN   
4        11     NaN      NaN   
...     ...     ...      ...   
3258  10861     NaN      NaN   
3259  10865     NaN      NaN   
3260  10868     NaN      NaN   
3261  10874     NaN      NaN   
3262  10875     NaN      NaN   

                                                   text  
0                    Just happened a terrible car crash  
1     Heard about #earthquake is different cities, s...  
2     there is a forest fire at spot pond, geese are...  
3              Apocalypse lighting. #Spokane #wildfires  
4         Typhoon Soudelor kills 28 in China and Taiwan  
...                                                 ...  
3258  EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...  
3259  Storm in RI worse than last hurricane. My city...  
3260  Green Line derailment in Chicago http://t.co/U...  
3261  MEG issues Hazardous Weather 

1. Lets remove the keywords and location columns. We only want to focus on the text and the predictions
2. Lets split some training data to validation dataset

In [22]:
SPLIT_PER = 2 #percentage of split for validation set 2 = 2%
split =  int(len(train_data_raw) * (SPLIT_PER/100))

dropped_train = train_data_raw.drop(columns=['id', 'keyword', 'location']) #drop the id, keyowrd and location columns from the train_data_raw

train_data = dropped_train[:-split]
valid_data = dropped_train[-split:]

assert train_data.shape == (len(train_data_raw) - split, 2)
#assert valid_data.shape == (split, 2)

In [23]:
print(train_data_raw.shape)
print("After dropping columns and spliting!")
print(train_data.shape, valid_data.shape)
 

(3263, 4)
After dropping columns and spliting!
(3198, 1) (65, 1)


## Lets tokenize the data

In [24]:
from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer('spacy', language='en_core_web_sm')
 
tokens = tokenizer("We are learning torchtext in AIT!")  #some test
tokens

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


['We', 'are', 'learning', 'torchtext', 'in', 'AIT', '!']

In [34]:
import pandas as pd
from torchtext.vocab import build_vocab_from_iterator

#def yield_tokens(data_iter):
  #for _, text in data_iter:            #loop through the data_iter, 
    # Mind that the data_iter in this case is pandas Dataframe
    #yield tokenizer(text) #remove this line and code here
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

specials = build_vocab_from_iterator(yield_tokens(train_data_raw), specials=['<unk>', '<pad>', '<bos>', '<eos>']) #create array of special tags for the vocab
vocab  = build_vocab_from_iterator(yield_tokens(train_data_raw), specials = specials, special_first=True)

#set_default_index of the vocab to unknown tag
vocab.set_default_index(vocab["<unk>"])

ValueError: ignored

In [ ]:
assert len(vocab) == 26442

In [ ]:
vocab_dict = vocab.get_stoi()
vocab_dict

In [35]:
from torchtext.vocab import FastText
fast_vectors = FastText(language='simple') ##Load fasttext with language=simple

.vector_cache/wiki.simple.vec: 293MB [00:08, 34.8MB/s]                           
100%|██████████| 111051/111051 [00:13<00:00, 8077.58it/s]


In [ ]:
fast_embedding = fast_vectors.get_vecs_by_tokens(vocab.get_itos()).to(device)

#since the fasttext  has 300 embedding
assert fast_embedding.shape == (len(vocab), 300)

In [ ]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1 ## Copy from the lab. Note that Something has to be changed

In [ ]:
text_pipeline("I love to play football")

In [ ]:
label_pipeline('1')

## To fit the padnas dataframe to DataLoader first we must wrap it as DataSet

In [ ]:

from torch.utils.data import Dataset

class PD_DATASET(Dataset):

    def __init__(self, dataframe):
        self.dataframe = dataframe
    
    def __len__(self):
        return len(self.dataframe)
        
    def __getitem__(self, idx):
        return self.dataframe.iloc[idx]

In [ ]:
train = 
valid = 
test = 

In [ ]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

pad_idx = vocab[''] ##get the pad index from the vocab

def collate_batch(batch):
    ## copy the collate_batch function from Professor's code. But it will not work right away
    #mind how the dataset that we use is structured (hint: columns)

In [ ]:
batch_size = 64

train_loader = DataLoader(train, batch_size=batch_size,
                              shuffle=True, collate_fn=collate_batch)
valid_loader = DataLoader(valid, batch_size=batch_size,
                              shuffle=True, collate_fn=collate_batch)

In [ ]:
next(iter(train_loader))

## First lets try CNN

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

## Get the Professor's code from  the lab to build the CNN model

class CNN(nn.Module):
    pass #replace this line with the respected code

In [ ]:
input_dim  = len(vocab)
emb_dim    =  #how many embedding does the fasttext have 
output_dim =  #how many classes do we have
dropout    = 0.5
n_filters  = 100 
filter_sizes = [3, 4, 5]

model = CNN(input_dim, emb_dim, output_dim, dropout, n_filters, filter_sizes).to(device)

In [ ]:
batch_size = 3
seq_len    = 50

 

In [ ]:
#explicitly initialize weights for better learning
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        nn.init.zeros_(m.bias)
    elif isinstance(m, (nn.Conv2d, nn.Conv2d)):
        for name, param in m.named_parameters():
            if 'bias' in name:
                nn.init.zeros_(param)
            elif 'weight' in name:
                nn.init.kaiming_normal_(param) 

In [ ]:
import torch.optim as optim

lr=1e-3

#training hyperparameters
optimizer = optim.SGD(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss() #combine softmax with cross entropy

In [ ]:
def accuracy(preds, y):
    
    predicted = torch.max(preds.data, 1)[1]
    batch_corr = (predicted == y).sum()
    acc = batch_corr / len(y)
    
    return acc

In [ ]:
def train(model, loader, optimizer, criterion, loader_length):
    #write the code to train the model

In [ ]:
def evaluate(model, loader, criterion, loader_length):
    #write the code to evaluate 

In [ ]:
train_loader_length = len(list(iter(train_loader)))
val_loader_length   = len(list(iter(valid_loader)))

In [ ]:
best_valid_loss = float('inf')
num_epochs      = 5

save_path = f'./models/{model.__class__.__name__}.pt'

train_losses = []
train_accs = []
valid_losses = []
valid_accs = []

for epoch in range(num_epochs):
        #write the code that starts the training, store the training and valid losses and accuracy
    #also print the time it took to train the model

In [ ]:
##Plot the training loss and the accuracy

# Lets Try the LSTM model

In [ ]:
train = PD_DATASET(train_data)
valid = PD_DATASET(valid_data)

In [ ]:
import torch.nn as nn

class LSTM(nn.Module):
    pass #replace this line with the real code

In [ ]:
input_dim  = len(vocab)
emb_dim    = #same as above
hidden_dim = #how many hidden dims do you want?
output_dim = #same as above
dropout    = 0.5
num_layers = 2
bidirectional = True 

lstm_model = LSTM(input_dim, emb_dim, hidden_dim, output_dim, num_layers, bidirectional, dropout).to(device)

In [ ]:
import torch.optim as optim

lr=1e-3

#training hyperparameters
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss() #

In [ ]:
def train(model, loader, optimizer, criterion, loader_length):
    #write the code to train the model 

In [ ]:
def evaluate(model, loader, criterion, loader_length):
    #write the code to evaluate the model

In [ ]:
train_loader_length = len(list(iter(train_loader)))
val_loader_length   = len(list(iter(valid_loader)))

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
best_valid_loss = float('inf')
num_epochs   = 5

save_path = f'./models/lstm_{model.__class__.__name__}.pt'

train_losses = []
train_accs = []
valid_losses = []
valid_accs = []

for epoch in range(num_epochs):
    #write the code that starts the training, store the training and valid losses and accuracy
    #also print the time it took to train the model

In [ ]:
##Plot the losses and accuracy over all epochs

## Conclusion
- Compare the two models on their time and accuracy. Which one do you think did well for the disaster classification task.
- How do you think we get better results in this dataset for classification.

#### Write your answer here